# Deploy Llama 2 on Amazon SageMaker with vLLM

---

[Llama 2](https://llama.meta.com/llama2/) are pretrained models trained on 2 trillion tokens with a 4k context length. Its fine-tuned chat models have been trained on over 1 million human annotations. Llama 2 has undergone internal and external adversarial testing across fine-tuned models to identify potential toxicity, bias, and other gaps in performance. To learn more about Llama 2 models, click [here](https://llama.meta.com/llama2/).

SageMaker has rolled out [vLLM container](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers) which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we combine the strengths of two powerful tools: [DJL](https://docs.djl.ai/) (Deep Java Library) for the serving framework and [vLLM](https://docs.vllm.ai/en/stable/) for distributed large language model inference. DJLServing, a high-performance universal model serving solution powered by DJL, handles the overall serving architecture.

In our setup, vLLM handles the core LLM inference tasks, leveraging its optimizations to achieve high performance and low latency. DJLServing manages the broader serving infrastructure, handling incoming requests, load balancing, and coordinating with vLLM for efficient inference.

This combination allows us to deploy the `Llama 2 7B` model across GPUs on the `ml.g5.12xlarge` instance with optimal resource utilization. vLLM's efficiencies in memory management and request handling enable us to serve this large model with improved throughput compared to traditional serving methods. To learn more about DJL, DJLServing, and vLLM you can refer to this [blog post](https://aws.amazon.com/blogs/machine-learning/boost-inference-performance-for-mixtral-and-llama-2-models-with-new-amazon-sagemaker-containers/).

---

<div class="alert alert-block alert-warning"> 

<b>NOTE:</b> Llama models are licensed under a bespoke commercial license that balances open access to the models with responsibility and protections in place to help address potential misuse. Their license allows for broad commercial use, as well as for developers to create and redistribute additional work on top of Llama models. For more details, their licenses can be found at [Meta Llama 2](https://llama.meta.com/license/) and [Meta Llama 3](https://llama.meta.com/llama3/license/).
</div>

## Requirements

1. Create an Amazon SageMaker Notebook Instance - [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
    - For Notebook Instance type, choose `ml.t3.medium`.
2. For Select Kernel, choose [conda_python3](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-prepare.html).
3. Install the required packages.

<div class="alert alert-block alert-info"> 

<b>NOTE:

- </b> For <a href="https://aws.amazon.com/sagemaker/studio/" target="_blank">Amazon SageMaker Studio</a>, select Kernel "<span style="color:green;">Python 3 (ipykernel)</span>".

- For <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html" target="_blank">Amazon SageMaker Studio Classic</a>, select Image "<span style="color:green;">Base Python 3.0</span>" and Kernel "<span style="color:green;">Python 3</span>".

</div>

To run this notebook you would need to install the following dependencies:

In [1]:
!pip install boto3==1.34.132 -qU --force --quiet --no-warn-conflicts
!pip install sagemaker==2.224.2 -qU --force --quiet --no-warn-conflicts

---

### Import libraries

In [2]:
import boto3
import json
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
print(sagemaker.__version__)

2.224.2


### Initialize parameters

In [4]:
# execution role for the endpoint
role = sagemaker.get_execution_role()

# sagemaker session for interacting with different AWS APIs
sess = sagemaker.session.Session()

# Region
region_name = sess._region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region_name}")

sagemaker role arn: arn:aws:iam::570598552974:role/txt2sql-SageMakerExecutionRole-PAgMr5TND4x0
sagemaker session region: us-east-1


### Image URI of the DJL Container

LMI DLCs offer a low-code interface that simplifies using state-of-the-art inference optimization techniques and hardware. LMI allows you to apply tensor parallelism; the latest efficient attention, batching, quantization, and memory management techniques; token streaming; and much more, by just requiring the model ID and optional model parameters. 

See available Large Model Inference DLC's [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers) and for more details [here](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/lmi/announcements/deepspeed-deprecation.md).

In [5]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi",
    region=region_name,
    version="0.28.0"
)
print(f"DCL Image going to be used is ---- > {inference_image_uri}")

DCL Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.28.0-lmi10.0.0-cu124


### Available Environment Variable Configurations

Here is a list of settings that we use in this configuration file:

- `HF_MODEL_ID`: The model id of a pretrained model hosted inside a model repository on [huggingface.co](https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. This is an optional setting and is not needed in the scenario where you are brining your own model. If you are getting your own model, you can include the URI of the Amazon S3 bucket that contains the model.
- `HF_TOKEN`: Some models on the HuggingFace Hub are gated and require permission from the owner to access. To deploy a gated model from the HuggingFace Hub using LMI, you must provide an [Access Token](https://huggingface.co/docs/hub/security-tokens) via this environment variable.
- `OPTION_ENGINE`: The engine for DJL to use. In this case, we intend to use [vLLM](https://docs.vllm.ai/en/stable/) and hence set it as **Python**.
- `OPTION_DTYPE`: The data type you plan to cast the model weights to. If not provided, LMI will use fp16.
- `OPTION_TGI_COMPAT`: To get the same response output as HuggingFace's Text Generation Inference, you can use the env `OPTION_TGI_COMPAT=true`.
- `OPTION_TASK`: The task used in Hugging Face for different pipelines. Default is text-generation. For further reading on DJL parameters on SageMaker, follow the [link](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/deepspeed_user_guide.html)
- `OPTION_ROLLING_BATCH`: Enables continuous batching (iteration level batching) with one of the supported backends. Available backends differ by container, see [Inference Library Configurations](https://docs.djl.ai/docs/serving/serving/docs/lmi/deployment_guide/configurations.html#inference-library-configuration) for mappings.
    - In the LMI Container:
        - to use vLLM, use `OPTION_ROLLING_BATCH=vllm`
        - to use lmi-dist, use `OPTION_ROLLING_BATCH=lmi-dist`
        - to use huggingface accelerate, use `OPTION_ROLLING_BATCH=auto` for text generation models, or option.rolling_batch=disable for non-text generation models.
- `TENSOR_PARALLEL_DEGREE`: Set to the number of GPU devices over which DeepSpeed needs to partition the model. This parameter also controls the no of workers per model which will be started up when DJL serving runs. Setting this to `max`, which will shard the model across all available GPUs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests.
- `OPTION_DEVICE_MAP`: The HuggingFace accelerate device_map to use.
- `OPTION_TRUST_REMOTE_CODE`: If the model artifacts contain custom modeling code, you should set this to true after validating the custom code is not malicious. If you are using a HuggingFace Hub model id, you should also specify HF_REVISION to ensure you are using artifacts and code that you have validated.

For more details on the configuration options and an exhaustive list, you can refer the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html) and [LMI Starting Guide](https://docs.djl.ai/docs/serving/serving/docs/lmi/user_guides/starting-guide.html)

## Create SageMaker endpoint

Here are some key differences between the available backends in LMI:

+ **LMI-Distributed Library** – This is the AWS framework to run inference with LLMs, inspired from OSS, to achieve the best possible latency and accuracy on the result. LMI-Dist employs optimized default configurations, such as GPU core counting, to ensure efficient performance and resource utilization.
+ **LMI vLLM** – This is the AWS backend implementation of the memory-efficient vLLM inference library

In [6]:
# Hugging Face Model Id
model_id = "meta-llama/Llama-2-7b-chat-hf"

# Environment variables
hf_token = "<REPLACE WITH YOUR TOKEN>" # Use for gated models
rolling_batch = "vllm" # "vllm", "lmi-dist"

env = {}
env['HF_MODEL_ID'] = model_id
env['OPTION_ROLLING_BATCH'] = rolling_batch
env['OPTION_DTYPE'] = "fp16"
env['OPTION_TGI_COMPAT'] = "true"

if rolling_batch != "lmi-dist":
    env['OPTION_ENGINE'] = "Python"
    env['OPTION_TASK'] = "text-generation"
    env['TENSOR_PARALLEL_DEGREE'] = "max"
    env['OPTION_DEVICE_MAP'] = "auto"
    # env['OPTION_TRUST_REMOTE_CODE'] = "true"
    
# Include HF token for gated models
if hf_token != "<REPLACE WITH YOUR TOKEN>":
    env['HF_TOKEN'] = hf_token
else:
    print("Llama models are gated, please add your HF token before you continue.")

In [7]:
# SageMaker Instance Type
instance_type = "ml.g5.12xlarge"

# Endpoint name
endpoint_name_prefix = "llama2-7b-chat-vllm"
endpoint_name = sagemaker.utils.name_from_base(endpoint_name_prefix)

print(f"instance_type: {instance_type}")
print(f"model_id: {model_id}")
print(f"endpoint_name: {endpoint_name}")

instance_type: ml.g5.12xlarge
model_id: meta-llama/Llama-2-7b-chat-hf
endpoint_name: llama2-7b-chat-vllm-2024-07-08-17-27-28-261


In [8]:
# Deploy model to an endpoint
model = sagemaker.Model(
    image_uri=inference_image_uri,
    role=role,
    env=env
)

In [9]:
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900,
)

--------------!

## Run inference and chat with the model

### Supported Inference Parameters

---
This model supports the following inference payload parameters:

* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches max_new_tokens. If specified, it must be a positive integer.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.

You may specify any subset of the parameters mentioned above while invoking an endpoint. 

---

### Inference using SageMaker SDK

In [10]:
# Initialize sagemaker client with the endpoint created in the prior step
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [11]:
prompt = """<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{message_prompt} [/INST] </s>""".format(
    system_prompt="You are a helpful assistant.",
    message_prompt="Building a website can be done in 10 simple steps:"
)

inputs = {
    "inputs": prompt,
    "parameters": {
        "temperature": 0.8,
        "top_p": 0.95,
        "max_new_tokens": 4000,
        "do_sample": False
    }
}
response = predictor.predict(inputs)
print(response[0]['generated_text'].strip())

01. Define Your Website's Purpose: Determine the purpose of your website, including the type of content, audience, and goals.

02. Choose a Domain Name: Select a unique and memorable domain name that reflects your website's purpose and is easy to spell and remember.

03. Select a Web Host: Choose a reliable and affordable web host that meets your website's needs, including storage space, bandwidth, and technical support.

04. Plan Your Website's Structure: Create a basic structure for your website, including the main pages and subpages, and organize them in a logical and intuitive manner.

05. Design Your Website: Create a visually appealing design for your website, including the layout, colors, fonts, and images, that reflects your brand and appeals to your target audience.

06. Write and Publish Content: Create high-quality, engaging and informative content for your website, including text, images, videos, and other media, that meets the needs of your target audience.

07. Set Up Nav

### Inference using Boto3 SDK

In [12]:
# Initialize sagemaker client with boto3 using the endpoint created from prior step
smr_client = boto3.client("sagemaker-runtime")

In [13]:
prompt = """<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{message_prompt} [/INST]""".format(
    system_prompt="You are a helpful assistant.",
    message_prompt="""what is the recipe of mayonnaise?"""
)

response = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": prompt,
            "parameters": {
                "temperature": 0.8,
                "top_p": 0.95,
                "max_new_tokens": 4000,
                "do_sample": False
            },
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

print(json.loads(response)[0]['generated_text'])

  Of course! Mayonnaise is a popular condiment made from a mixture of egg yolks, oil, vinegar or lemon juice, and seasonings. Here's a basic recipe for homemade mayonnaise:

Ingredients:

* 2 egg yolks
* 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
* 1 tablespoon (15 ml) vinegar or lemon juice
* 1/2 teaspoon (2.5 ml) salt
* 1/4 teaspoon (1.25 ml) sugar (optional)
* 1/4 teaspoon (1.25 ml) mustard powder (optional)

Instructions:

1. In a small bowl, whisk together the egg yolks and salt until well combined.
2. Slowly pour in the oil while continuously whisking the mixture. You can do this by hand with a whisk or use an electric mixer on low speed.
3. Once you've added about half of the oil, add the vinegar or lemon juice and continue whisking until the mixture thickens and emulsifies. This should take about 5-7 minutes.
4. Taste and adjust the seasoning as needed. If the mayonnaise is too thick, add a little water. If it's too thin, add more oil.
5. Cover the bowl w

## Conclusion
In this post, we demonstrated how to use SageMaker large model inference containers to host Codestral 22B. We used DeepSpeed’s model parallel techniques with multiple GPUs on a single SageMaker machine learning instance. 

## Clean Up

In [14]:
# Delete the endpoint
sess.delete_endpoint(endpoint_name)

In [15]:
# In case the end point failed we still want to delete the model
sess.delete_endpoint_config(endpoint_name)
model.delete_model()